In [9]:
import xml.etree.ElementTree as et
import glob
import re
import pandas as pd
from collections import OrderedDict
import sqlalchemy as sa
    
res = []

def compute_xml_data(lst,elem, func, level=0):
    func(lst,elem,level)
    for child in elem.getchildren():
        compute_xml_data(lst,child, func, level+1)

def gather_level(lst,elem,level):
    lst.append(('-'*level+elem.tag, elem.text))

xml_files = glob.glob('./*.xml')

for xml_file in xml_files:
    lst = []
    root = et.parse(xml_file)
    compute_xml_data(lst,root.getroot(), gather_level)
    res.append(lst)

def get_table_names(xml_data_list): 
    table_names = []
    for t in xml_data_list:
        if t[0]:
            if re.match(r'(-{http://webservices.irb.com/}\w)', t[0], re.IGNORECASE):
                name = t[0][30:].strip()
                table_names.append(name)
    return table_names
                
tables = get_table_names(res[0])

for xml_file in xml_files:
    lst = []
    root = et.parse(xml_file)
    compute_xml_data(lst,root.getroot(), gather_level)

# build all the dataframe for the xml
def build_xml_dfs(xmls):
    personID_tag = '-{http://webservices.irb.com/}PersonID'
    personnel_profile_tag = '{http://webservices.irb.com/}PersonnelProfile'
    teamsRepresented_tag = '-{http://webservices.irb.com/}TeamsRepresented'
    team_tag = '--{http://webservices.irb.com/}Team'
    testBreakdown_tag = '-{http://webservices.irb.com/}TestBreakdown'
    eventAppearances_tag = '-{http://webservices.irb.com/}EventAppearances'
    appearance_tag = '--{http://webservices.irb.com/}Appearance'
    
    personID_idx = [xml[0] for xml in xmls].index(personID_tag)
    personID = xmls[personID_idx][1]
    
    personnel_profile_list = []
    teamsRepresented_list = []
    testBreakdown_list = []
    eventAppearances_list = []
    
    for t in xmls:
        if t[0] == personnel_profile_tag: 
            personnel_profile_start_idx = [xml[0] for xml in xmls].index(personnel_profile_tag)
            teamsRepresented_start_idx = [xml[0] for xml in xmls].index(teamsRepresented_tag)
            testBreakdown_start_idx = [xml[0] for xml in xmls].index(testBreakdown_tag)
            eventAppearances_start_idx = [xml[0] for xml in xmls].index(eventAppearances_tag)

            for m in xmls[personnel_profile_start_idx+1:teamsRepresented_start_idx]:
                if m[0] is not teamsRepresented_tag:
                    personnel_profile_list.append((m[0][30:].strip(),m[1]))
                else:
                    continue
            for o in xmls[teamsRepresented_start_idx+1:testBreakdown_start_idx]:
                if o[0] == team_tag:
                    sub_list = []
                    sub_list.append(('PersonID',personID))
                    teamsRepresented_list.append(sub_list)
                elif o[0] is not testBreakdown_tag:
                    sub_list.append((o[0][32:].strip(),o[1]))
                else:
                    continue
            for t1 in xmls[testBreakdown_start_idx+1:eventAppearances_start_idx]:
                if t1[0] == team_tag:
                    sub_list = []
                    sub_list.append(('PersonID',personID))
                    testBreakdown_list.append(sub_list)
                elif t1[0] is not eventAppearances_tag:
                    sub_list.append((t1[0][32:].strip(),t1[1]))
                else:
                    continue
            for time_line in xmls[eventAppearances_start_idx+1:]:
                if time_line[0] == appearance_tag:
                    sub_list = []
                    sub_list.append(('PersonID',personID))
                    eventAppearances_list.append(sub_list)
                elif time_line[0]:
                    sub_list.append((time_line[0][32:].strip(),time_line[1]))
                else:
                    break                 
               
    records_dict = {'personnel_profile': personnel_profile_list, 'teamsRepresented': teamsRepresented_list, 'testBreakdown': testBreakdown_list, 'eventAppearances':eventAppearances_list}
    personnel_profile_df = pd.DataFrame.from_records(personnel_profile_list).transpose()
    personnel_profile_df = personnel_profile_df.rename(columns=personnel_profile_df.iloc[0]).drop(personnel_profile_df.index[0])

    teamsRepresented_df = [pd.DataFrame.from_records(lst).transpose() for lst in teamsRepresented_list if lst]
    teamsRepresented_df = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in teamsRepresented_df]

    testBreakdown_df = [pd.DataFrame.from_records(lst).transpose() for lst in testBreakdown_list if lst]
    testBreakdown_df = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in testBreakdown_df ]
    
    eventAppearances_df = [pd.DataFrame.from_records(lst).transpose() for lst in eventAppearances_list if lst]
    eventAppearances_df = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in eventAppearances_df]
     
    personnel_profile_dfs_merged = personnel_profile_df
    if teamsRepresented_df: 
        teamsRepresented_dfs_merged = pd.concat(teamsRepresented_df)
    else:
        teamsRepresented_dfs_merged = pd.DataFrame()
    if testBreakdown_df:
        testBreakdown_dfs_merged = pd.concat(testBreakdown_df)
    else:
        testBreakdown_dfs_merged = pd.DataFrame()
    if eventAppearances_df:
        eventAppearances_dfs_merged = pd.concat(eventAppearances_df)
    else:
        eventAppearances_dfs_merged = pd.DataFrame()
    
    return {'personnel_profile':personnel_profile_dfs_merged,
            'teamsRepresented':teamsRepresented_dfs_merged,
            'testBreakdown':testBreakdown_dfs_merged,
            'eventAppearances':eventAppearances_dfs_merged}    
            
all_xml_dfs = []
personnel_profile_dfs_list = []
teamsRepresented_dfs_list = []
testBreakdown_dfs_list = []
eventAppearances_dfs_list = []

for xml_data in res:
    all_xml_dfs.append(build_xml_dfs(xml_data))
    
    
for df_dict in all_xml_dfs:
    for k,v in df_dict.items():
        if k == 'personnel_profile':
            personnel_profile_dfs_list.append(v)
        if k == 'teamsRepresented':
            teamsRepresented_dfs_list.append(v)
        if k == 'testBreakdown':
            testBreakdown_dfs_list.append(v)
        if k == 'eventAppearances':
            eventAppearances_dfs_list.append(v)
            
if personnel_profile_dfs_list:
    personnel_profile_dfs_merged_all = pd.concat(personnel_profile_dfs_list)
else:
    personnel_profile_dfs_merged_all = pd.DataFrame()
if teamsRepresented_dfs_list:
    teamsRepresented_dfs_merged_all = pd.concat(teamsRepresented_dfs_list)
else: 
    teamsRepresented_dfs_merged_all = pd.DataFrame()
if testBreakdown_dfs_list:
    testBreakdown_dfs_merged_all = pd.concat(testBreakdown_dfs_list)
else:
    testBreakdown_dfs_merged_all = pd.DataFrame()
if eventAppearances_dfs_list:
    eventAppearances_dfs_merged_all = pd.concat(eventAppearances_dfs_list)
else: 
    eventAppearances_dfs_merged_all = pd.DataFrame()

# load dataframe into Postgres
def write_data_to_sql(df, table_name):        
    try:
        engine = sa.create_engine(connection_string, echo=True);
        # add a table id?
        # df["TableId"] = tableId;
        if not df.empty:
            df.to_sql(tablename, engine, if_exists='append', index=False)       
    except Exception as e:
        print(e)

personnel_profile_dfs_merged_all.to_csv('./playerprofile_personnel_profile.csv',index=False)
teamsRepresented_dfs_merged_all.to_csv('./playerprofile_teamsRepresented.csv',index=False)
testBreakdown_dfs_merged_all.to_csv('./playerprofile_testBreakdown.csv',index=False)
eventAppearances_dfs_merged_all.to_csv('./playerprofile_eventAppearances.csv',index=False)
print('Completed!')


[('PersonID', '23925'), ('PersonName', 'Pedro Leal'), ('DateOfBirth', '1984-04-28T00:00:00.000'), ('PlaceOfBirth', 'Lisbon'), ('Height', '170'), ('Weight', '75'), ('Position', None)]
[('PersonID', '26114'), ('PersonName', 'Hiroki Yoshida'), ('DateOfBirth', '1981-11-16T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '26435'), ('PersonName', 'Phil Mack'), ('DateOfBirth', '1985-09-18T00:00:00.000'), ('PlaceOfBirth', 'Victoria'), ('Height', '170'), ('Weight', '77'), ('Position', None)]
[('PersonID', '26802'), ('PersonName', 'Scott Wight'), ('DateOfBirth', '1985-11-05T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '27056'), ('PersonName', 'Julien Candelon'), ('DateOfBirth', '1980-07-08T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '28585'), ('PersonName', 'Vladimir Ostroushko'), ('DateOfBirth', '1986-09-3

[('PersonID', '48529'), ('PersonName', 'Santiago Alvarez Fourcade'), ('DateOfBirth', '1994-02-17T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '188'), ('Weight', '89'), ('Position', None)]
[('PersonID', '48550'), ('PersonName', 'Kwagga Smith'), ('DateOfBirth', '1993-06-11T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '181'), ('Weight', '85'), ('Position', None)]
[('PersonID', '48565'), ('PersonName', 'Cheslin Kolbe'), ('DateOfBirth', '1993-10-23T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '170'), ('Weight', '74'), ('Position', None)]
[('PersonID', '48567'), ('PersonName', 'Justin Geduld'), ('DateOfBirth', '1993-10-01T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '175'), ('Weight', '70'), ('Position', None)]
[('PersonID', '48582'), ('PersonName', 'Seabelo Senatla'), ('DateOfBirth', '1993-02-10T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '176'), ('Weight', '79'), ('Position', None)]
[('PersonID', '48638'), ('PersonName', 'Axel Muller'), ('DateOfBirth', '1993-

[('PersonID', '52287'), ('PersonName', 'Ivan Korotkov'), ('DateOfBirth', '1990-10-09T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '52291'), ('PersonName', 'Vladislav Lazarenko'), ('DateOfBirth', '1989-10-02T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '52312'), ('PersonName', 'James Stannard'), ('DateOfBirth', '1983-02-21T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '52331'), ('PersonName', 'Ryno Benjamin'), ('DateOfBirth', '1983-08-03T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '52341'), ('PersonName', 'Kyle Brown'), ('DateOfBirth', '1987-02-06T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '52350'), ('PersonName', 'Chris Dry'), ('DateOfBirth', '1988-02-13T00:00:00.0

[('PersonID', '55572'), ('PersonName', 'Kitione Taliga'), ('DateOfBirth', '1993-04-21T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '55574'), ('PersonName', 'Alexandre Gracbling'), ('DateOfBirth', '1996-04-08T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '55575'), ('PersonName', 'Tiago Fernandes'), ('DateOfBirth', '1996-12-05T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '187'), ('Weight', '95'), ('Position', None)]
[('PersonID', '55576'), ('PersonName', 'Fetufou Setu Laga'), ('DateOfBirth', '1994-08-22T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '55577'), ('PersonName', 'Jay Saena'), ('DateOfBirth', '1993-10-29T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '55646'), ('PersonName', 'Robbie Nairn'), ('DateOfBirth', '1997-02-18T0

[('PersonID', '42556'), ('PersonName', 'Pama Fou'), ('DateOfBirth', '1990-09-06T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '185'), ('Weight', '100'), ('Position', None)]
[('PersonID', '42663'), ('PersonName', 'Carl Murray'), ('DateOfBirth', '1985-03-27T00:00:00.000'), ('PlaceOfBirth', 'Whangarei'), ('Height', '181'), ('Weight', '90'), ('Position', None)]
[('PersonID', '42988'), ('PersonName', 'Mike Fuailefau'), ('DateOfBirth', '1992-03-20T00:00:00.000'), ('PlaceOfBirth', 'Victoria'), ('Height', '185'), ('Weight', '99'), ('Position', None)]
[('PersonID', '42991'), ('PersonName', 'Pat Kay'), ('DateOfBirth', '1993-09-19T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '182'), ('Weight', '92'), ('Position', None)]
[('PersonID', '43088'), ('PersonName', 'Madison Hughes'), ('DateOfBirth', '1992-10-26T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '173'), ('Weight', '80'), ('Position', None)]
[('PersonID', '43114'), ('PersonName', 'Kuda Chiwanza'), ('DateOfBirth', '1991-05-29T00

...

[('PersonID', '55216'), ('PersonName', 'Luke Bradley'), ('DateOfBirth', '1996-10-05T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '55641'), ('PersonName', 'George Horne'), ('DateOfBirth', '1995-05-12T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '55692'), ('PersonName', 'Lloyd Lewis'), ('DateOfBirth', '1996-09-05T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '55783'), ('PersonName', 'Jamie Farndale'), ('DateOfBirth', '1994-02-21T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '55784'), ('PersonName', 'Hugh Blake'), ('DateOfBirth', '1992-09-10T00:00:00.000'), ('PlaceOfBirth', 'Tauranga'), ('Height', '187'), ('Weight', '102'), ('Position', None)]
[('PersonID', '55785'), ('PersonName', 'Glenn Bryce'), ('DateOfBirth', '1991-06-07T00:00:00.0

[('PersonID', '59973'), ('PersonName', 'Naoki Motomura'), ('DateOfBirth', '1992-04-11T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '59990'), ('PersonName', 'Elisapeta Alofipo'), ('DateOfBirth', '1997-12-11T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '60003'), ('PersonName', 'Jean-teiva Jacquelin'), ('DateOfBirth', '1994-04-22T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '60222'), ('PersonName', 'Jared Douglas'), ('DateOfBirth', '1992-06-24T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '60223'), ('PersonName', 'Pierre Popelin'), ('DateOfBirth', '1995-06-23T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '60224'), ('PersonName', 'Jone Naikabula'), ('DateOfBirth', '1994-04

[('PersonID', '47437'), ('PersonName', 'Charlie Hayter'), ('DateOfBirth', '1988-12-10T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '188'), ('Weight', '100'), ('Position', None)]
[('PersonID', '47467'), ('PersonName', 'Jaike Carter'), ('DateOfBirth', '1986-09-11T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '182'), ('Weight', '92'), ('Position', None)]
[('PersonID', '48058'), ('PersonName', 'Joe Webber'), ('DateOfBirth', '1993-08-27T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '184'), ('Weight', '92'), ('Position', None)]
[('PersonID', '48360'), ('PersonName', 'Justin Douglas'), ('DateOfBirth', '1994-04-05T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '187'), ('Weight', '93'), ('Position', None)]
[('PersonID', '48658'), ('PersonName', 'Mike Teo'), ('DateOfBirth', '1993-07-23T00:00:00.000'), ('PlaceOfBirth', None), ('Height', '175'), ('Weight', '92'), ('Position', None)]
[('PersonID', '48721'), ('PersonName', 'James Benjamin'), ('DateOfBirth', '1994-02-21T00:00:00.0

[('PersonID', '59395'), ('PersonName', 'Ryan Oosthuizen'), ('DateOfBirth', '1995-05-22T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '59403'), ('PersonName', 'Thomas Glyn Williams'), ('DateOfBirth', '1996-07-30T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '59410'), ('PersonName', 'Setareki Bituniyata'), ('DateOfBirth', '1995-08-12T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '59411'), ('PersonName', 'Kalione Nasoko'), ('DateOfBirth', '1990-12-02T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '59424'), ('PersonName', 'Jonmoon Han'), ('DateOfBirth', '1996-07-07T00:00:00.000'), ('PlaceOfBirth', None), ('Height', None), ('Weight', None), ('Position', None)]
[('PersonID', '59426'), ('PersonName', 'Dallas Tatana'), ('DateOfBirth', '1991-08

Completed!


In [2]:
import xml.etree.ElementTree as et
import glob
import re
import pandas as pd
from collections import OrderedDict
import sqlalchemy as sa
    
res = []

def compute_xml_data(elem, func, level=0):
    func(elem,level)
    for child in elem.getchildren():
        compute_xml_data(child, func, level+1)

def gather_level(elem,level):
    print('-'*level+elem.tag, elem.text)
#     res.append(('-'*level+elem.tag, elem.text))

xml_files = glob.glob('./*.xml')
xml_file = xml_files[0]
root = et.parse(xml_file) 
xml_data = compute_xml_data(root.getroot(), gather_level)

def get_table_names(xml_data_list): 
    table_names = []
    for t in xml_data_list:
        if t[0]:
            if re.match(r'(-{http://webservices.irb.com/}\w)', t[0], re.IGNORECASE):
                name = t[0][30:].strip()
                table_names.append(name)
    return table_names

# print(res)

print(get_table_names(res))

# compute_xml_data(root.getroot(), gather_level)


{http://webservices.irb.com/}PersonnelProfile 
  
-{http://webservices.irb.com/}PersonID 26114
-{http://webservices.irb.com/}PersonName Hiroki Yoshida
-{http://webservices.irb.com/}DateOfBirth 1981-11-16T00:00:00.000
-{http://webservices.irb.com/}PlaceOfBirth None
-{http://webservices.irb.com/}Height None
-{http://webservices.irb.com/}Weight None
-{http://webservices.irb.com/}Position None
-{http://webservices.irb.com/}TeamsRepresented 
    
--{http://webservices.irb.com/}Team 
      
---{http://webservices.irb.com/}TeamID 2423
---{http://webservices.irb.com/}TeamName Japan 7s, Japan - Mens Sevens (Country)
---{http://webservices.irb.com/}RoleName Player
---{http://webservices.irb.com/}Country Japan
---{http://webservices.irb.com/}TeamType Country
--{http://webservices.irb.com/}Team 
      
---{http://webservices.irb.com/}TeamID 49
---{http://webservices.irb.com/}TeamName Japan, Japan - Mens (Country)
---{http://webservices.irb.com/}RoleName Player
---{http://webservices.irb.com/}Count